In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [2]:
def get_html(url)->BeautifulSoup:
    driver = webdriver.Chrome()
    driver.get(url)
    try:
        element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '.cxLeGg'))
        WebDriverWait(driver, 15).until(element_present)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    except:
        soup = BeautifulSoup("<body><tagerroramericanas>ERRORCARREGAMENTO404</tagerroramericanas></body>", 'html.parser')
    finally:
        driver.close()
        driver.quit()
        return soup

In [3]:
def extrair_dados(soups, Listacards):
    """Essa função extrai todas as informações"""
    try:
        for soup in soups:
            page_card = soup.body.find("div", {"class": "cxLeGg"})
            card_list = page_card.find_all("div", {"class": "fsViFX"})
            for item in card_list:
                descricao = item.h3.text
                Total_Reviews = item.span
                try:
                    Total_Reviews = int(Total_Reviews.getText().split()[0])
                except:
                    Total_Reviews = 0
                Preco = item.find("span", {"class":"ijXFcq"}) 
                try:
                    Preco = Preco.text.split()[1].replace(".","").replace(",",".")
                except:
                    Preco = 0
                
                url = "https://www.americanas.com.br" + item.a.get("href")
                
                cards = {"Descrição": descricao,  "TotalReviews": Total_Reviews, "Preço": float(Preco), "Url": url, "Loja": "Americanas" }
                Listacards.append(cards)
    except:
        Listacards = []

In [4]:
def salvar_data_set(Listacards, pesquisa):
    if len(Listacards) > 0:
        dataset = pd.DataFrame(Listacards)
        dataset.to_csv(f'output/dataset_{pesquisa}.csv'.replace(' ','_'), index = False, encoding = 'utf-8-sig', sep=";")
        print(f"Dataset salvo com sucesso : output/dataset_{pesquisa}.csv'")
    else:
        print("Erro ao salvar o dataSET")

In [5]:
def search_produto(pesquisa):
    pesquisa_path = pesquisa.replace(' ','-')
    urls = [f"https://www.americanas.com.br/busca/{pesquisa_path}?limit=50&offset={x}" for x in range(0, 350, 50)]
    # LISTA DE TODOS OS HTLMS RETORNADOS
    soups = []
    
    
    for url in urls:
        # ESSE WHILE NOS GARANTE QUE SEMPRE SERÁ RETORNADO O HTML CORRETO
        while True: 
            soup = get_html(url)
            try:
                if "ERRORCARREGAMENTO404" not in soup.text:
                    break
            except:
                pass
            time.sleep(1)
        soups.append(soup)
       
    # Criando uma lista para salvar todos os cards
    Listacards = []
    #extraindo as informações
    extrair_dados(soups=soups, Listacards=Listacards )
    #Salvando dataSet
    salvar_data_set(Listacards=Listacards, pesquisa=pesquisa)
    

In [6]:
search_produto("notebook")

Dataset salvo com sucesso : output/dataset_notebook.csv'


In [7]:
search_produto("smart tv")


Dataset salvo com sucesso : output/dataset_smart tv.csv'


In [8]:
search_produto("smartphone")

Dataset salvo com sucesso : output/dataset_smartphone.csv'


In [10]:
search_produto("geladeira")

Dataset salvo com sucesso : output/dataset_geladeira.csv'
